## 03 数据增强

在上一个 Notebook 中，我们分析了数据集的基本情况，发现数据集中存在明显的不平衡现象，尤其是中性情感样本数量较少。为了缓解这种不平衡性对模型训练的影响，我们将对中性情感样本进行数据增强处理。

下面介绍两种 NLP 任务中常用的数据增强方法。

In [10]:
import pandas as pd
from pandas import DataFrame

# 读取数据
df = pd.read_csv('data/weibo_features.csv', encoding='utf-8-sig')

# 进行至此，可以删除掉 DataFrame 中的冗余字段，如文本长度 text_length
df = df.drop(columns=["text_length"])

print(df["sentiment_type"].value_counts())

neutral_data = df[df["sentiment_type"] == "中立"]

neutral_data.sample(5)

sentiment_type
积极    167778
消极    124334
中立     66472
Name: count, dtype: int64


,text,sentiment_type,sentiment_polarity
256351,每次打完底在定妆之前都有一步重要工作 从脸上往下摘猫毛,中立,0
151206,来买饮料的路上就这么水灵灵地又买上了,中立,0
284991,连续三天吃喝tims,中立,0
47061,9点多睡到11点多 然后看文到现在 就是不咋困,中立,0
102841,为四点以后的生活 保持50的分享欲,中立,0


### 1. 回译（Back-translation）
回译是一种常用的数据增强方法，通过将文本翻译成另一种语言再翻译回来，从而生成新的文本。这个过程可以增加数据的多样性，帮助模型更好地泛化。

例如：

- 原始文本：今天北京温度在 20 度左右。
- 翻译成英文：The temperature in Beijing is around 20°C today.
- 再翻译回中文：北京今天气温大约二十摄氏度。

通过回译，我们可以生成与原始文本意思相近但表述不同的新样本。生成风格自然的新增句子。

这里，我们使用 HuggingFace 上的 Helsinki-NLP/opus-mt-zh-en 和 Helsinki-NLP/opus-mt-en-zh 翻译模型完成回译任务。这两个模型是 OPUS-MT(Open Parallel Corpus - Machine Translation) 项目的重要组成部分，专门针对中英互译任务进行了深度优化。

由于模型位于 Huggingface 服务器上，若通过国内网络访问模型来处理数据，可能会受限于网络状况导致效果不佳。因此，我们先将模型从 Huggingface 下载到本地，然后再加载本地模型进行离线翻译。

两个翻译模型已事先下载至项目根目录下的 models 文件夹中，见 `models/opus-mt-zh-en` 和 `models/opus-mt-en-zh`。

```python


In [11]:
from transformers import MarianMTModel, MarianTokenizer

def load_model(src, tgt):
    model_name = f'models/opus-mt-{src}-{tgt}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return model, tokenizer

def translate(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs)
    translated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return translated_texts

def back_translate(sentence):
    en = translate([sentence], model_zh2en, tok_zh2en)[0]
    zh = translate([en], model_en2zh, tok_en2zh)[0]
    return zh

model_zh2en, tok_zh2en = load_model('zh', 'en')
model_en2zh, tok_en2zh = load_model('en', 'zh')


我们从中性情感样本中抽取$50\%$的数据进行回译增强，生成新的中性样本。得到的增强样本数量为
$$66472 \times 50\% = 33236$$

In [12]:
texts_to_translate = neutral_data["text"].sample(frac=0.5, random_state=42).to_list()

texts_to_translate[:10]

['你们俩 谈恋爱不准耽误学习！',
 '真假？不是说深圳是最后的行程吗',
 '黄礼志，像翻盖手机一样的女人。',
 '2025年',
 '一夜花开',
 'hola！',
 '深呼吸深呼吸',
 '抖抖发了12条，其他的草稿发在微博不过分吧',
 '心理治疗的本质，不过是一种自律的工具。 斯科特派克少有人走的路',
 '往事有底片为证']

为了提高回译效率，我们使用批量回译的方法处理数据。

简单的实验表明，批量回译相较于逐条回译，效率可提升约 2 倍

In [13]:
import pandas as pd
import os
import torch
from tqdm.auto import tqdm

def batch_back_translate(sentences, saved_path="data/texts_back_translated.csv", 
                                   batch_size=32, max_length=512, save_every=100):
    """
    增量保存版本的批量回译函数
    
    Args:
        sentences: 要翻译的句子列表
        saved_path: CSV保存路径
        batch_size: 批处理大小
        max_length: 最大序列长度
        save_every: 每处理多少条数据保存一次
    
    Returns:
        DataFrame包含所有回译结果
    """
    # 检查是否已有部分结果
    if os.path.exists(saved_path):
        existing_df = pd.read_csv(saved_path, encoding='utf-8-sig')
        processed_count = len(existing_df)
        print(f"发现已处理 {processed_count} 条数据，从第 {processed_count + 1} 条开始...")
        sentences = sentences[processed_count:]
        if not sentences:
            print("所有数据已处理完成！")
            return existing_df
    else:
        existing_df = pd.DataFrame(columns=['original_text', 'augmented_text', 'intermediate_english'])
        processed_count = 0
    
    # 创建保存目录
    os.makedirs(os.path.dirname(saved_path), exist_ok=True)
    
    all_results = []
    
    print(f"开始处理剩余 {len(sentences)} 条文本...")
    
    # 分批处理
    for i in tqdm(range(0, len(sentences), batch_size), desc="增量回译中"):
        batch = sentences[i:i + batch_size]
        
        try:
            # 执行回译
            inputs_zh = tok_zh2en(batch, return_tensors="pt", padding=True, 
                                 truncation=True, max_length=max_length)
            
            with torch.no_grad():
                en_outputs = model_zh2en.generate(**inputs_zh, max_length=max_length, 
                                                num_beams=4, early_stopping=True)
                en_texts = tok_zh2en.batch_decode(en_outputs, skip_special_tokens=True)
            
            inputs_en = tok_en2zh(en_texts, return_tensors="pt", padding=True, 
                                 truncation=True, max_length=max_length)
            
            with torch.no_grad():
                zh_outputs = model_en2zh.generate(**inputs_en, max_length=max_length,
                                                num_beams=4, early_stopping=True)
                zh_texts = tok_en2zh.batch_decode(zh_outputs, skip_special_tokens=True)
            
            # 收集批次结果
            batch_results = []
            for j, (original, translated, english) in enumerate(zip(batch, zh_texts, en_texts)):
                batch_results.append({
                    'original_text': original,
                    'augmented_text': translated,
                    'intermediate_english': english
                })
            
            all_results.extend(batch_results)
            
            # 定期保存结果
            if len(all_results) >= save_every or i + batch_size >= len(sentences):
                # 合并现有数据和新数据
                new_df = pd.DataFrame(all_results)
                combined_df = pd.concat([existing_df, new_df], ignore_index=True)
                
                # 保存到文件
                combined_df.to_csv(saved_path, index=False, encoding='utf-8-sig')
                
                print(f"已保存 {len(combined_df)} 条数据到 {saved_path}")
                
                # 更新现有数据框
                existing_df = combined_df
                all_results = []  # 清空临时结果
            
            # 清理GPU内存
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                
        except Exception as e:
            print(f"处理批次时出错: {e}")
            continue

    print(f"所有数据处理完成！最终结果: {len(existing_df)} 条")
    return existing_df


In [14]:
translated_results = batch_back_translate(texts_to_translate)
print(f"回译结果大小：{len(translated_results)}")
translated_results = translated_results.dropna()
print(f"删除缺失值后回译结果大小：{len(translated_results)}")

发现已处理 33236 条数据，从第 33237 条开始...
所有数据已处理完成！
回译结果大小：33236
删除缺失值后回译结果大小：33234


从抽取的回译结果中可以看到，微博作为社交平台，用户发布的内容往往较为口语化，往往并不严格按照书面语的规范组织文字，这导致了回译生成的文本可能出现语义偏差、结构混乱、信息缺失等问题。

然而，尽管回译生成的文本在语义上存在细微偏差，但在情感倾向上与原文本基本保持一致，因此，这些回译生成的文本仍然可以作为有效的中性情感样本用于数据增强，帮助模型更好地学习中性情感的表达方式。

In [15]:
translated_data = DataFrame([
    (text, "中立", 0) for text in translated_results["augmented_text"]
], columns=["text", "sentiment_type", "sentiment_polarity"])

# 将回译数据加入到原始 DataFrame 中
df_aug = pd.concat([df, translated_data], ignore_index=True)
    
# 打乱
df_aug = df_aug.sample(frac=1).reset_index(drop=True)

print(f"回译前数据集中性样本数量: {len(neutral_data)}")
print(f"回译后数据集中性样本数量: {len(df_aug[df_aug['sentiment_type'] == '中立'])}")

回译前数据集中性样本数量: 66472
回译后数据集中性样本数量: 99706


最后，我们查看数据增强后数据集中标签的分布情况：

In [19]:
type_counts = df_aug["sentiment_type"].value_counts()
print(type_counts)

sentiment_type
积极    167778
消极    124334
中立     99706
Name: count, dtype: int64


和 `02Feature Engineering and Data Analysis.ipynb` 一样，计算最大类/最小类 比率 IR 与变异系数 CV：
> - $IR \leq 1.5$ → 非常平衡
> - $1.5 < IR \leq 3$ → 轻度不平衡
> - $IR > 3$ → 明显不平衡
> - $IR > 10$ → 高度不平衡，需要采样或加权

In [20]:
type_counts_before = df["sentiment_type"].value_counts()

ir_before = type_counts_before.max() / type_counts_before.min()
ir = type_counts.max() / type_counts.min()

print(f"数据增强前 IR = {ir_before:.2f}")
print(f"数据增强后 IR = {ir:.2f}")

数据增强前 IR = 2.52
数据增强后 IR = 1.68


从 IR 可以看出不平衡程度有所减小。

> - $CV \leq 0.1$ → 非常平衡
> - $0.1 < CV \leq 0.3$ → 可接受
> - $CV > 0.3$ → 明显不平衡

In [21]:
cv_before = type_counts_before.std() / type_counts_before.mean()
cv = type_counts.std() / type_counts.mean()

print(f"数据增强前 CV = {cv_before:.2f}")
print(f"数据增强后 CV = {cv:.2f}")

数据增强前 CV = 0.43
数据增强后 CV = 0.26


从 CV 同样可以看到不平衡程度有所减弱。

在这个 Notebook 中，我们主要为了缓解数据集中的类别不平衡问题，采用了回译的方法对中性情感样本进行了数据增强。通过回译生成了大量新的中性样本，丰富了数据集的多样性，有助于提升模型在中性情感识别上的表现。

然而，正如 IR 和 CV 值所示，数据集仍然存在一定程度的不平衡问题。在后续的模型训练过程中，我们将考虑结合其他技术手段，如加权损失函数，进一步缓解类别不平衡对模型性能的影响。

In [23]:
df_aug.to_csv('data/weibo_augmented.csv', index=False, encoding='utf-8-sig')